In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import missingno as msno
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from clf_evaluation import *
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

%matplotlib inline

rc('font', family='Malgun Gothic')
sns.set_style('whitegrid')

In [5]:
df = pd.read_csv('../data/train.csv/train.csv')
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [6]:
df_ohe = pd.get_dummies(df[['Geography', 'Gender']])
pre_ohe = df.drop(['Geography', 'Gender'], axis=1)
data = pd.concat([pre_ohe, df_ohe], axis=1)
columns_order = [col for col in data.columns if col != 'Exited']
data = data[columns_order + ['Exited']]
data.head()

,id,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
0,0,15674932,Okwudilichukwu,668,33.0,3,0.00,2,1.0,0.0,181449.97,True,False,False,False,True,0
1,1,15749177,Okwudiliolisa,627,33.0,1,0.00,2,1.0,1.0,49503.50,True,False,False,False,True,0
2,2,15694510,Hsueh,678,40.0,10,0.00,2,1.0,0.0,184866.69,True,False,False,False,True,0
3,3,15741417,Kao,581,34.0,2,148882.54,1,1.0,1.0,84560.88,True,False,False,False,True,0
4,4,15766172,Chiemenam,716,33.0,5,0.00,2,1.0,1.0,15068.83,False,False,True,False,True,0


In [15]:
X = data.drop(['Exited', 'id', 'CustomerId', 'Surname'], axis=1)
y = data['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=13)
X_ovr, y_ovr = smote.fit_resample(X_train, y_train)

In [16]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X_ovr.shape, y_ovr.shape)

(132027, 13) (33007, 13) (132027,) (33007,)
(208122, 13) (208122,)


## 로지스틱 회귀

1. Exited - 불균형 상태

In [17]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_proba = lr.predict_proba(X_test)
lr_pred = lr.predict(X_test)

print_eval(y_test, lr_pred, lr_proba[:, 1])

Confusion Matrix
[[25139   913]
 [ 6085   870]]
Accuracy: 0.788 Precision: 0.4879 Recall: 0.1251
F1: 0.1991 ROC AUC: 0.7152


2. Exited - "class_weight = 'balanced'"

In [14]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
lr_proba = lr.predict_proba(X_test)
lr_pred = lr.predict(X_test)

print_eval(y_test, lr_pred, lr_proba[:, 1])

Confusion Matrix
[[17859  8191]
 [ 2245  4712]]
Accuracy: 0.6838 Precision: 0.3652 Recall: 0.6773
F1: 0.4745 ROC AUC: 0.7348


- accuracy, precision이 하락하는 대신 recall이 크게 개선되어 auc_score가 소폭 상승
- 타겟 데이터 불균형의 해소로 모델의 성능이 개선

> Recall, Precision의 의미
- recall : 실제 이탈 고객 중 모델이 얼마나 많은 이탈 고객을 올바르게 예측했는가
    - 이탈 가능성이 있는 고객을 놓치는 것이 비즈니스에 큰 손실을 야기할 때 중요
- precision : 모델이 이탈한다고 예측한 고객 중 실제로 이탈한 고객의 비율
    - 이탈 하지 않을 고객을 이탈할 것으로 예측하는 비율이 높다면, 이탈 고객 방지를 위한 마케팅 비용이 낭비될 수 있음
- auc score : 모델이 이탈 고객과 비이탈 고객을 얼마나 잘 구분하는가
    - 모델의 전반적인 성능 평가에 유용

3. oversampling - SMOTE

In [20]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(X_ovr, y_ovr)
lr_proba = lr.predict_proba(X_test)
lr_pred = lr.predict(X_test)

print_eval(y_test, lr_pred, lr_proba[:, 1])

Confusion Matrix
[[17867  8185]
 [ 2223  4732]]
Accuracy: 0.6847 Precision: 0.3663 Recall: 0.6804
F1: 0.4762 ROC AUC: 0.7373


- 'class_weight'만 설정했을 때와 성능상 큰 차이가 보이진 않음

## Decision Tree

1. Exited - 불균형 상태

In [22]:
dt = DecisionTreeClassifier(max_depth=3, random_state=13)
dt.fit(X_train, y_train)
dt_proba = dt.predict_proba(X_test)
dt_pred = dt.predict(X_test)

print_eval(y_test, dt_pred, dt_proba[:, 1])

Confusion Matrix
[[25309   743]
 [ 4181  2774]]
Accuracy: 0.8508 Precision: 0.7887 Recall: 0.3988
F1: 0.5298 ROC AUC: 0.8448


2. Exited - class_weight 사용

In [23]:
dt = DecisionTreeClassifier(max_depth=3, random_state=13, class_weight='balanced')
dt.fit(X_train, y_train)
dt_proba = dt.predict_proba(X_test)
dt_pred = dt.predict(X_test)

print_eval(y_test, dt_pred, dt_proba[:, 1])

Confusion Matrix
[[20168  5884]
 [ 1601  5354]]
Accuracy: 0.7732 Precision: 0.4764 Recall: 0.7698
F1: 0.5886 ROC AUC: 0.8448


- auc score를 통해 종합적으로 평가할 수 있는 모델의 성능은 로지스틱 회귀에 비해 향상됨

> class_weight에 따른 성능 차이
- class_weight 미사용: 더 높은 정밀도와 낮은 재현율. 모델이 실제 이탈 고객을 예측하는 데 보수적. 모델은 이탈할 것으로 확신이 드는 경우에만 이탈을 예측하고 이로 인해 잘못된 이탈 예측(거짓 양성)이 적지만, 실제 이탈 고객 중 많은 수를 놓치고 있음

- class_weight='balanced' 사용: 재현율이 크게 증가하고 정밀도가 감소. 'balanced' 옵션은 불균형한 클래스 가중치를 조정하여 소수 클래스(이탈 고객)에 더 많은 중요도를 부여함. 결과적으로 모델은 이탈할 가능성이 있는 고객을 더 많이 식별하려 하지만, 이 과정에서 비이탈 고객을 잘못 이탈 고객으로 분류하는 경우가 증가. 

-> 비즈니스 목적에 따라 어떤 모델을 선택할 것인지 달라질 수 있음

3. 오버샘플링

In [34]:
dt = DecisionTreeClassifier(max_depth=3, random_state=13)
dt.fit(X_ovr, y_ovr)
dt_proba = dt.predict_proba(X_test)
dt_pred = dt.predict(X_test)

print_eval(y_test, dt_pred, dt_proba[:, 1])

Confusion Matrix
[[20168  5884]
 [ 1601  5354]]
Accuracy: 0.7732 Precision: 0.4764 Recall: 0.7698
F1: 0.5886 ROC AUC: 0.8426


- 로지스틱 회귀와 마찬가지로 오버샘플링 한 것과 class_weight를 사용한 것의 차이는 없음

## 랜덤포레스트

1. Exited - 불균형 상태

In [27]:
rf = RandomForestClassifier(random_state=13, max_depth=3)
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_test)
rf_pred = rf.predict(X_test)

print_eval(y_test, rf_pred, rf_proba[:, 1])

Confusion Matrix
[[25908   144]
 [ 6080   875]]
Accuracy: 0.8114 Precision: 0.8587 Recall: 0.1258
F1: 0.2195 ROC AUC: 0.8637


2. Exited - class_weight 사용

In [30]:
rf = RandomForestClassifier(random_state=13, max_depth=3, class_weight='balanced')
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_test)
rf_pred = rf.predict(X_test)

print_eval(y_test, rf_pred, rf_proba[:, 1])

Confusion Matrix
[[20367  5685]
 [ 1647  5308]]
Accuracy: 0.7779 Precision: 0.4829 Recall: 0.7632
F1: 0.5915 ROC AUC: 0.8573


- class_weight 사용시 auc score 소폭 하락.
    - recall은 크게 상승했지만, accuracy와 precision이 하락. 
    - 소수 클래스에 과도한 가중치 부여로 모델의 일반화 능력 저하
    - class_weight 조정으로 일부 트리가 소수 클래스에 과도하게 편향되었을 가능성

3. 오버샘플링

In [33]:
rf = RandomForestClassifier(random_state=13, max_depth=3)
rf.fit(X_ovr, y_ovr)
rf_proba = rf.predict_proba(X_test)
rf_pred = rf.predict(X_test)

print_eval(y_test, rf_pred, rf_proba[:, 1])

Confusion Matrix
[[21902  4150]
 [ 2246  4709]]
Accuracy: 0.8062 Precision: 0.5315 Recall: 0.6771
F1: 0.5955 ROC AUC: 0.8521


## LGBM

1. Exited - 불균형 상태

In [35]:
lgbm = LGBMClassifier(random_state=13, max_depth=3)
lgbm.fit(X_train, y_train)
lgbm_proba = lgbm.predict_proba(X_test)
lgbm_pred = lgbm.predict(X_test)

print_eval(y_test, lgbm_pred, lgbm_proba[:, 1])

[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002219 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 864
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Confusion Matrix
[[24835  1217]
 [ 3185  3770]]
Accuracy: 0.8666 Precision: 0.756 Recall: 0.5421
F1: 0.6314 ROC AUC: 0.8898


2. Exited - class_weight

In [36]:
lgbm = LGBMClassifier(random_state=13, max_depth=3, class_weight='balanced')
lgbm.fit(X_train, y_train)
lgbm_proba = lgbm.predict_proba(X_test)
lgbm_pred = lgbm.predict(X_test)

print_eval(y_test, lgbm_pred, lgbm_proba[:, 1])

[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002603 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 864
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Confusion Matrix
[[21006  5046]
 [ 1379  5576]]
Accuracy: 0.8053 Precision: 0.5249 Recall: 0.8017
F1: 0.6345 ROC AUC: 0.8896


> 데이터 불균형과 precision <-> recall 관계
- 정밀도(Precision) 하락: class_weight='balanced'를 사용하면 모델은 소수 클래스의 예측에 더 많은 중점을 두고, 모델은 실제로는 음성 클래스인 샘플들을 양성 클래스로 잘못 분류할 가능성이 높아짐. 모델이 양성으로 예측한 샘플들 중 실제로 양성인 샘플의 비율이 감소
- 재현율(Recall) 상승: 소수 클래스에 더 많은 가중치를 부여함으로써 모델은 실제 양성 클래스 샘플을 놓치지 않으려고 하고, 결과적으로 실제 양성 클래스인 샘플들 중 모델이 양성으로 올바르게 예측한 비율이 증가

-> 데이터 불균형 문제 해결을 위한 시도는 precision과 recall의 트레이드 오프 관계

3. 오버샘플링

In [37]:
lgbm = LGBMClassifier(random_state=13, max_depth=3)
lgbm.fit(X_ovr, y_ovr)
lgbm_proba = lgbm.predict_proba(X_test)
lgbm_pred = lgbm.predict(X_test)

print_eval(y_test, lgbm_pred, lgbm_proba[:, 1])

[LightGBM] [Info] Number of positive: 104061, number of negative: 104061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007412 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1556
[LightGBM] [Info] Number of data points in the train set: 208122, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Confusion Matrix
[[24079  1973]
 [ 2621  4334]]
Accuracy: 0.8608 Precision: 0.6872 Recall: 0.6231
F1: 0.6536 ROC AUC: 0.8876


> LGBM에서 class_weight과 오버 샘플링 사용 비교
- class_weight을 사용한 경우, 
    - 재현율이 높게 측정됨. 이는 모델이 실제 이탈 고객을를 놓치지 않는 데 효과적임을 의미
    - 그러나 이 모델은 낮은 정밀도를 가지고 있어, 이탈 고객이 아닌 경우에도 이탈 고객으로 잘못 예측하는 경우가 많음
- SMOTE를 사용한 오버샘플링을 한 경우, 
    - 더 높은 정확도와 정밀도를 보이지만, 재현율이 첫 번째 모델보다 낮음
    - 더 적은 수의 거짓 양성을 생성하지만, 양성 케이스를 놓칠 가능성이 더 높음(실제 이탈 고객을 놓칠 가능성이 더 높음)

-> 어떤 모델이 더 나은가?
- 비즈니스 요구에 따라 다름: 
    - 이탈 예측과 같은 경우, 재현율이 더 중요할 수 있음. 이 경우, class_weight를 사용한 모델을 선택
    - 비용과 위험 고려: 잘못된 이탈 예측이 마케팅 비용이나 다른 부정적인 결과(고객 불편)를 초래할 수 있다면, 오버 샘플링을 사용한 모델을 선택